In [84]:
import pandas as pd
import numpy as np

df_credit = pd.read_csv('credits.csv')
df_rating = pd.read_csv('ratings.csv')
df_movie = pd.read_csv('movies_metadata.csv')
df_keywords = pd.read_csv('keywords.csv')


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [162]:
df_movie = df_movie[['original_title', 'id', 'title', 'adult', 'belongs_to_collection', 
                     'budget', 'genres', 'original_language', 'popularity',
                     'production_companies', 'production_countries', 'revenue', 
                     'runtime', 'spoken_languages', 'status', 'vote_average',
                     'vote_count']]

df_rating = df_rating[['movieId', 'rating']]



In [267]:
def convertType(x, typ):
    try:
        return typ(x)
    except ValueError:
        return np.NaN

df_movie['genre_cleaned'] = df_movie['genres'].apply(lambda x: [d['name'] for d in ast.literal_eval(x)])

df_movie['is_collection'] = df_movie['belongs_to_collection'].isnull()
df_movie['is_collection'] = df_movie['is_collection'].apply(lambda x: 1 if x==True else 0)
df_movie['is_collection']

df_movie['adult'] = df_movie['adult'].apply(lambda x: 1 if x==True else 0)
df_movie['popularity'] = df_movie['popularity'].apply(lambda x: convertType(x, float))

df_movie = df_movie[df_movie['revenue'] != 0]

df_movie_cleaned = df_movie[['revenue', 'is_collection', 'adult', 'popularity', 'vote_average', 'vote_count', 'runtime']]
df_movie_cleaned.astype({
    'is_collection': 'int32',
    'adult': 'int32',
    'popularity': 'float',
    'revenue': 'float',
    'vote_average': 'float',
    'vote_count': 'float',
    'runtime': 'float'
})

,revenue,is_collection,adult,popularity,vote_average,vote_count,runtime
0,373554033.0,0,0,21.946943,7.7,5415.0,81.0
1,262797249.0,1,0,17.015539,6.9,2413.0,104.0
3,81452156.0,1,0,3.859495,6.1,34.0,127.0
4,76578911.0,0,0,8.387519,5.7,173.0,106.0
5,187436818.0,1,0,17.924927,7.7,1886.0,170.0
...,...,...,...,...,...,...,...
45399,3.0,1,0,0.201582,6.0,4.0,0.0
45401,50656.0,1,0,0.436028,6.3,3.0,110.0
45409,1328612.0,1,0,0.903061,5.8,6.0,100.0
45412,1268793.0,1,0,0.121844,4.0,3.0,107.0


In [268]:
# Drop rows with any NAN
df_movie_cleaned = df_movie_cleaned.dropna()

x = df_movie_cleaned.iloc[:, 1:].values
y = df_movie_cleaned.iloc[:, 0].values

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train, y_train)

# Make Prediction
df_result = pd.DataFrame()
y_predicted = regressor.predict(x_test)
combined = np.concatenate((np.vstack((y_test)), x_test), axis=1)
df_result = pd.DataFrame(data=combined, columns=df_movie_cleaned.columns)
df_result['predicted_revenue'] = y_predicted

df_result['difference'] = (df_result['predicted_revenue']-df_result['revenue'])/df_result['revenue']


In [271]:
df_result
regressor.coef_

array([-4.75562027e+07, -1.77026959e-06,  7.58489279e+05, -9.50475212e+06,
        9.41940427e+04,  4.65239166e+05])

In [165]:
# import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder# instantiate OneHotEncoder
# df_movie['is_collection']
# df_movie['genre_cleaned'] 
# ohe = OneHotEncoder().fit(df_movie['genre_cleaned']) 
# categorical_features = boolean mask for categorical columns
# sparse = False output an array not sparse matrix

In [116]:
import re

df_credit
import ast
keywords = ast.literal_eval(df_keywords.iloc[0]['keywords'])
keywords
# re.split("},", df_credit.iloc[0]['cast'].strip(']['))
# df_credit.iloc[0]['cast'].strip('][')
# pd.to_numeric(df_movie['popularity'])
# df_movie.iloc[35587]

[{'id': 931, 'name': 'jealousy'},
 {'id': 4290, 'name': 'toy'},
 {'id': 5202, 'name': 'boy'},
 {'id': 6054, 'name': 'friendship'},
 {'id': 9713, 'name': 'friends'},
 {'id': 9823, 'name': 'rivalry'},
 {'id': 165503, 'name': 'boy next door'},
 {'id': 170722, 'name': 'new toy'},
 {'id': 187065, 'name': 'toy comes to life'}]

In [141]:
categorical_feature_mask = df_movie.dtypes==object
categorical_feature_mask

adult                     True
belongs_to_collection     True
budget                    True
genres                    True
homepage                  True
id                        True
imdb_id                   True
original_language         True
original_title            True
overview                  True
popularity                True
poster_path               True
production_companies      True
production_countries      True
release_date              True
revenue                  False
runtime                  False
spoken_languages          True
status                    True
tagline                   True
title                     True
video                     True
vote_average             False
vote_count               False
genre_cleaned             True
is_collection            False
dtype: bool

In [147]:
# import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder# instantiate OneHotEncoder
df_movie['genre_cleaned'] = df_movie['genres'].apply(lambda x: [d['name'] for d in ast.literal_eval(x)])

df_movie['is_collection'] = df_movie['belongs_to_collection'].isnull()
df_movie['is_collection'] = df_movie['is_collection'].apply(lambda x: 1 if x==True else 0)
df_movie['is_collection']

df_movie['adult'] = df_movie['adult'].apply(lambda x: 1 if x==True else 0)

df_movie['popularity'] = 
df_movie.astype({
    'is_collection': 'int32',
    'adult': 'int32',
    'popularity': 'float'
})

# df_movie['is_collection']
# df_movie['genre_cleaned'] 
# ohe = OneHotEncoder().fit(df_movie['genre_cleaned']) 
# categorical_features = boolean mask for categorical columns
# sparse = False output an array not sparse matrix

ValueError: could not convert string to float: 'Beware Of Frost Bites'

In [260]:
df_result['difference'].mean()

92232.81691898152

In [261]:
df_result

,revenue,is_collection,adult,popularity,vote_average,predicted_revenue,difference
0,18.0,1.0,0.0,1.145806,3.1,-2.741782e+07,-1.523213e+06
1,401421.0,1.0,0.0,2.903563,4.9,4.826253e+06,1.102292e+01
2,73121.0,1.0,0.0,2.796081,6.1,2.007817e+07,2.735883e+02
3,11100000.0,1.0,0.0,1.970054,6.7,2.393474e+07,1.156283e+00
4,170362582.0,1.0,0.0,10.388183,5.5,4.924560e+07,-7.109365e-01
...,...,...,...,...,...,...,...
1477,50000000.0,1.0,0.0,2.474398,6.7,2.639635e+07,-4.720729e-01
1478,9800000.0,0.0,0.0,1.258022,6.1,1.125443e+08,1.048411e+01
1479,5535405.0,0.0,0.0,11.304979,7.7,1.826170e+08,3.199073e+01
1480,57059003.0,0.0,0.0,8.997507,6.2,1.516340e+08,1.657495e+00
